In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [ ]:
bets = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
bets = bets[:-1]
bets

,Away,Home,is_B2B_First_Away,is_B2B_First_Home,is_B2B_Second_Away,is_B2B_Second_Home,Away_Prob_Naive,Home_Prob_Naive,Home_Prob_Adjusted,Away_Prob_Adjusted,...,Home_Prob_Odds,Away_Prob_Odds,Home_Prob_Diff,Away_Prob_Diff,Home_Bet_Size,Away_Bet_Size,Home_Bet,Away_Bet,Potential_Payoff,Date
0,New York Knicks,Portland Trailblazers,0,0,0,0,0.410397,0.253672,0.380594,0.619406,...,0.387597,0.652778,-0.007003,-0.033371,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-12
1,San Antonio Spurs,New Orleans Pelicans,0,0,1,0,0.436023,0.253041,0.383889,0.616111,...,0.701493,0.340136,-0.317603,0.275975,0.000000,0.041823,0.000000,4658.673264,9037.826133,2022-02-12
2,Sacramento Kings,Washington Wizards,0,0,0,0,0.251726,0.232051,0.548901,0.451099,...,0.454545,0.586777,0.094355,-0.135678,0.017298,0.000000,1926.880917,0.000000,2312.257100,2022-02-12
3,Memphis Grizzlies,Charlotte Hornets,0,0,0,1,0.598545,0.472978,0.413332,0.586668,...,0.289855,0.750000,0.123477,-0.163332,0.017388,0.000000,1936.808882,0.000000,4745.181760,2022-02-12
4,Cleveland Cavaliers,Philadelphia 76ers,0,0,1,1,0.599922,0.466752,0.427549,0.572451,...,0.624060,0.420168,-0.196512,0.152283,0.000000,0.026263,0.000000,2925.482992,4037.166529,2022-02-12
5,Denver Nuggets,Toronto Raptors,0,0,1,0,0.670816,0.545997,0.467625,0.532375,...,0.621212,0.420168,-0.153587,0.112207,0.000000,0.019352,0.000000,2155.582320,2974.703602,2022-02-12
6,Oklahoma City Thunder,Chicago Bulls,0,0,1,1,0.193722,0.363328,0.816310,0.183690,...,0.870130,0.169492,-0.053820,0.014199,0.000000,0.001710,0.000000,190.435350,933.133216,2022-02-12
7,Brooklyn Nets,Miami Heat,0,0,0,0,0.361205,0.585889,0.828772,0.171228,...,0.855072,0.185185,-0.026301,-0.013957,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-12
8,Los Angeles Clippers,Dallas Mavericks,0,0,0,0,0.231973,0.517185,0.904861,0.095139,...,0.729730,0.312500,0.175131,-0.217361,0.064799,0.000000,7217.937880,0.000000,2673.310326,2022-02-12
9,Los Angeles Lakers,Golden State Warriors,0,0,0,0,0.415872,0.606519,0.793498,0.206502,...,0.746835,0.294118,0.046662,-0.087615,0.018432,0.000000,2053.101808,0.000000,695.966715,2022-02-12


In [23]:
results = pd.read_csv('In_Season/Data/results_tracker.csv', index_col = 0)
results

In [17]:
odds_df

,Home_Team,Away_Team,Home_Odds,Away_Odds,Home_Prob,Away_Prob
0,Portland Trailblazers,New York Knicks,158.0,-188.0,38.759690,65.277778
1,Washington Wizards,Sacramento Kings,126.0,-148.0,44.247788,59.677419
2,New Orleans Pelicans,San Antonio Spurs,-235.0,194.0,70.149254,34.013605
3,Charlotte Hornets,Memphis Grizzlies,245.0,-300.0,28.985507,75.000000
4,Toronto Raptors,Denver Nuggets,-164.0,138.0,62.121212,42.016807
5,Philadelphia 76ers,Cleveland Cavaliers,-166.0,138.0,62.406015,42.016807
6,Chicago Bulls,Oklahoma City Thunder,-670.0,490.0,87.012987,16.949153
7,Miami Heat,Brooklyn Nets,-590.0,440.0,85.507246,18.518519
8,Dallas Mavericks,Los Angeles Clippers,-270.0,220.0,72.972973,31.250000
9,Golden State Warriors,Los Angeles Lakers,-295.0,240.0,74.683544,29.411765
